Initialize MNIST data

Please upload the files in `func_simMNIST` to the `/home` directory

In [3]:
%pip install avalanche-lib
%cd /home   
%run download_data.py
%run generate_txt.py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.0/811.0 KB 51.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 89.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.2/517.2 KB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.4/532.4 KB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.7/455.7 KB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 KB 22.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 21.7 MB/s eta 0:00:00
  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7409 sha256=c46207120

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting /root/.avalanche/data/mnist/TensorMNIST/raw/train-images-idx3-ubyte.gz to /root/.avalanche/data/mnist/TensorMNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting /root/.avalanche/data/mnist/TensorMNIST/raw/train-labels-idx1-ubyte.gz to /root/.avalanche/data/mnist/TensorMNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting /root/.avalanche/data/mnist/TensorMNIST/raw/t10k-images-idx3-ubyte.gz to /root/.avalanche/data/mnist/TensorMNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting /root/.avalanche/data/mnist/TensorMNIST/raw/t10k-labels-idx1-ubyte.gz to /root/.avalanche/data/mnist/TensorMNIST/raw



In [4]:
import torch 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [9]:
"""
calculate the functional similarity between task 1 (distinguish digits 0 and 1) and others
"""
import torch
import os
import torch.nn as nn
import torch.optim as optim
import numpy as np


from model import MLP
from util import trainES,get_MNIST

# ------------------------------------ step 0/5 : initialise hyper-parameters ------------------------------------
basic_task = 0  # count from 0
experience = 5
train_bs = 128
test_bs = 128
lr_init = 0.001
max_epoch = 100
run_times = 10
patience = 20
hidden_dim = 400
# 是否使用GPU
no_cuda = False
use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
kwargs = {"num_workers":1, "pin_memory":True} if use_cuda else {}

fun_score = np.zeros((run_times, 4))
for run in range(run_times):
    print("run time: {}".format(run+1))
# ------------------------------------ step 1/5 : load data------------------------------------
    train_stream, test_stream = get_MNIST()
# ------------------------------------ step 2/5 : define network-------------------------------
    model = MLP(out_dim=2, in_channel=1,hidden_dim=hidden_dim)
    # ToDo
    # 后续应该在网络定义中添加全职初始化函数
    # ------------------------------------ step 3/5 : define loss function and optimization ------------------------
    criterion = nn.CrossEntropyLoss()  
    optimizer = optim.SGD(model.parameters(), lr=lr_init)  
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1) 
    # ------------------------------------ step 4/5 : training --------------------------------------------------
    # training basic task
    train_data = train_stream[basic_task]
    test_data = test_stream[basic_task]
    model,_ , _ ,avg_valid_losses , _ = trainES(train_data,test_data,model,criterion,optimizer,max_epoch,device,patience = 20)
    basic_loss = avg_valid_losses[-1]
    print("basic loss:{:.4}".format(basic_loss))
    # save task 1
    PATH = "./"
    trained_model_path = os.path.join(PATH, 'model4sim.pth')
    torch.save(model.state_dict(), trained_model_path)

    # pop the src data from train_stream and test_stream
    train_stream.pop(basic_task)
    test_stream.pop(basic_task)
    # for other tasks , load task 1
    for j, probe_data in enumerate(train_stream):
        # load old task's model
        trained_model = MLP(out_dim=2, in_channel=1,hidden_dim=hidden_dim)
        trained_model.load_state_dict(torch.load(trained_model_path))
        criterion = nn.CrossEntropyLoss()  
        optimizer = optim.SGD(trained_model.parameters(), lr=lr_init, momentum=0.9, dampening=0.1)  
        train_data = probe_data
        # load other tasks
        for k, batch in enumerate(train_data):  
            x_train, y_train = batch[0].to(device), batch[1].to(device)
            trained_model.to(device)
            y_pred = trained_model(x_train)
            loss = criterion(y_pred, y_train)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # calculate the functional similarity (fun_score) at last steps
            if k == 0:
                print(loss.item())
                fun_score[run, j] = (1 - (loss.item() / basic_loss.item()))
                break
    print(fun_score[run, :])   
fun_score_mean = np.mean(fun_score, axis=0)
fun_score_std = np.std(fun_score, axis=0)
print(fun_score_mean)
print(fun_score_std)


run time: 1
[  0/100]  train_loss: 0.68420 valid_loss: 0.66388 train_acc: 0.64421 valid_acc: 0.89242
Validation loss decreased (inf --> 0.663880).  Saving model ...
[  1/100]  train_loss: 0.64655 valid_loss: 0.62603 train_acc: 0.77613 valid_acc: 0.94322
Validation loss decreased (0.663880 --> 0.626029).  Saving model ...
[  2/100]  train_loss: 0.60812 valid_loss: 0.58431 train_acc: 0.84336 valid_acc: 0.96107
Validation loss decreased (0.626029 --> 0.584310).  Saving model ...
[  3/100]  train_loss: 0.56368 valid_loss: 0.53676 train_acc: 0.88009 valid_acc: 0.97012
Validation loss decreased (0.584310 --> 0.536758).  Saving model ...
[  4/100]  train_loss: 0.51420 valid_loss: 0.48316 train_acc: 0.90259 valid_acc: 0.97563
Validation loss decreased (0.536758 --> 0.483159).  Saving model ...
[  5/100]  train_loss: 0.45957 valid_loss: 0.42536 train_acc: 0.91774 valid_acc: 0.97931
Validation loss decreased (0.483159 --> 0.425363).  Saving model ...
[  6/100]  train_loss: 0.40142 valid_loss: 0.

the mean and std of functional similarity, task 1 and 3 have the cloest inter-task relatedness

In [10]:
fun_score_mean 

array([-324.84379433, -339.04257578, -259.87654385, -367.75891402])

In [11]:
fun_score_std

array([44.70298679, 45.44025834, 47.25835676, 33.33945755])

In [13]:
"""
learn tasks in 2 phases
"""
import torch
import os
import torch.nn as nn
import torch.optim as optim
import numpy as np


from model import MLP
from util import trainES,get_MNIST,test
from metrics import compute_acc_fgt

# ------------------------------------ step 0/5 : initalzation------------------------------------
basic_task = 0
experience = 5
train_bs = 128
test_bs = 128
lr_init = 0.001
max_epoch = 30
run_times = 10
patience = 20
hidden_dim = 400

accuracy_list1 = [] # multiple run
accuracy_list2 = []
accuracy_list3 = []
accuracy_list4 = []

no_cuda = False
use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
kwargs = {"num_workers":1, "pin_memory":True} if use_cuda else {}

fun_score = np.zeros((run_times, 4))
for run in range(run_times):
    print("run time: {}".format(run+1))
# ------------------------------------ step 1/5 : load data------------------------------------
    train_stream, test_stream = get_MNIST()
# ------------------------------------ step 2/5 : define network------------------------------------
    model = MLP(out_dim=2, in_channel=1,hidden_dim=hidden_dim)

    # ------------------------------------ step 3/5 : define loss function and optim ------------------------------------=
    criterion = nn.CrossEntropyLoss()  
    optimizer = optim.SGD(model.parameters(), lr=lr_init)  
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)  
    # ------------------------------------ step 4/5 : training --------------------------------------------------
    # training basic task
    basic_task_data = train_stream[basic_task]
    basic_task_test_data = test_stream[basic_task]
    model, _, _, _, _ = trainES(basic_task_data, basic_task_test_data, model, criterion, optimizer, max_epoch, device,
                                patience)
    # setting stage 1 matrix
    acc_array1 = np.zeros((4, 2))
    # testing basic task
    _, acc_array1[:, 0] = test(test_stream[basic_task], model, criterion, device)
    # pop basic task
    train_stream.pop(basic_task)
    test_stream.pop(basic_task)
    # test other tasks except basic task
    for i, probe_data in enumerate(test_stream):
        with torch.no_grad():
            _, acc_array1[i, 1] = test(probe_data, model, criterion, device)
    # save task 1
    PATH = "./"
    trained_model_path = os.path.join(PATH, 'modle4acc_fgt.pth')
    torch.save(model.state_dict(), trained_model_path)
    # setting stage 2 matrix
    acc_array2 = np.zeros((4, 2))
    for j, (train_data, test_data) in enumerate(zip(train_stream, test_stream)):
        print("task {} starting...".format(j))
        # load old task's model
        # trained_model = MLP(out_dim=2, in_channel=in_channel)
        trained_model = MLP(out_dim=2, in_channel=1,hidden_dim=hidden_dim)
        trained_model.load_state_dict(torch.load(trained_model_path))
        criterion = nn.CrossEntropyLoss()  
        optimizer = optim.SGD(trained_model.parameters(), lr=lr_init, momentum=0.9, dampening=0.1)  
        # training other tasks
        trained_model, _, _, _, _ = trainES(train_data, test_data, trained_model, criterion, optimizer, max_epoch,
                                            device, patience)
        # test model on basic task and task j
        with torch.no_grad():
            _, acc_array2[j, 0] = test(basic_task_test_data, trained_model, criterion, device)
            _, acc_array2[j, 1] = test(test_stream[j], trained_model, criterion, device)
    # computing avg_acc and CF
    accuracy_list1.append([acc_array1[0, :], acc_array2[0, :]])
    accuracy_list2.append([acc_array1[1, :], acc_array2[1, :]])
    accuracy_list3.append([acc_array1[2, :], acc_array2[2, :]])
    accuracy_list4.append([acc_array1[3, :], acc_array2[3, :]])

accuracy_array1 = np.array(accuracy_list1)
accuracy_array2 = np.array(accuracy_list2)
accuracy_array3 = np.array(accuracy_list3)
accuracy_array4 = np.array(accuracy_list4)

avg_end_acc, avg_end_fgt, avg_acc = compute_acc_fgt(accuracy_array1)
print('----------- Avg_End_Acc {} Avg_End_Fgt {} Avg_Acc {}-----------'.format(avg_end_acc, avg_end_fgt, avg_acc))

avg_end_acc, avg_end_fgt, avg_acc = compute_acc_fgt(accuracy_array2)
print('----------- Avg_End_Acc {} Avg_End_Fgt {} Avg_Acc {}-----------'.format(avg_end_acc, avg_end_fgt, avg_acc))
avg_end_acc, avg_end_fgt, avg_acc = compute_acc_fgt(accuracy_array3)
print('----------- Avg_End_Acc {} Avg_End_Fgt {} Avg_Acc {}-----------'.format(avg_end_acc, avg_end_fgt, avg_acc))
avg_end_acc, avg_end_fgt, avg_acc = compute_acc_fgt(accuracy_array4)
print('----------- Avg_End_Acc {} Avg_End_Fgt {} Avg_Acc {}-----------'.format(avg_end_acc, avg_end_fgt, avg_acc))


run time: 1
[ 0/30]  train_loss: 0.67886 valid_loss: 0.66167 train_acc: 0.62124 valid_acc: 0.86464
Validation loss decreased (inf --> 0.661668).  Saving model ...
[ 1/30]  train_loss: 0.64490 valid_loss: 0.62623 train_acc: 0.74147 valid_acc: 0.92634
Validation loss decreased (0.661668 --> 0.626231).  Saving model ...
[ 2/30]  train_loss: 0.60933 valid_loss: 0.58660 train_acc: 0.81462 valid_acc: 0.94890
Validation loss decreased (0.626231 --> 0.586601).  Saving model ...
[ 3/30]  train_loss: 0.56728 valid_loss: 0.54142 train_acc: 0.85723 valid_acc: 0.96064
Validation loss decreased (0.586601 --> 0.541416).  Saving model ...
[ 4/30]  train_loss: 0.52007 valid_loss: 0.49052 train_acc: 0.88403 valid_acc: 0.96787
Validation loss decreased (0.541416 --> 0.490519).  Saving model ...
[ 5/30]  train_loss: 0.46803 valid_loss: 0.43490 train_acc: 0.90228 valid_acc: 0.97261
Validation loss decreased (0.490519 --> 0.434896).  Saving model ...
[ 6/30]  train_loss: 0.41216 valid_loss: 0.37711 train_ac